# Step 14: Function Invocation Filters

This notebook demonstrates how to add filters to track and log function calls in Semantic Kernel.

In [ ]:
## Import Required Libraries

In [ ]:
import sys
from pathlib import Path
from typing import Awaitable, Callable

# Add parent directory to import custom plugins
sys.path.insert(0, str(Path.cwd().parent))

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.filters import FunctionInvocationContext, FilterTypes

from plugins.datetime import DateTimePlugin
from plugins.location import LocationPlugin

## Define the Logger Filter

This filter intercepts function calls and logs them before and after execution.

In [ ]:
async def logger_filter(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Logs function invocations before and after execution."""
    print(f"Before: {context.function.plugin_name}.{context.function.name}")
    await next(context)
    print(f"After: {context.function.plugin_name}.{context.function.name}")

## Setup the Kernel with Filters

In [ ]:
kernel = Kernel()
azure_chat_completion = AzureChatCompletion()

kernel.add_service(azure_chat_completion)

# Add custom plugins
kernel.add_plugin(LocationPlugin(), plugin_name="location_plugin")
kernel.add_plugin(DateTimePlugin(), plugin_name="time_plugin")

# Add the function invocation filter
kernel.add_filter(FilterTypes.FUNCTION_INVOCATION, logger_filter)

# Configure execution settings
execution_settings = AzureChatPromptExecutionSettings(
    function_choice_behavior=FunctionChoiceBehavior.Auto()
)

## Run the Chat Loop

Try asking:
- "What time is it?"
- "Where am I?"
- "What's the current date and time?"

Notice how the filter logs each function call!

In [ ]:
async def main():
    history = ChatHistory()
    
    print("Chat started! Type 'exit' to quit.")
    print("Try: 'What time is it?' or 'Where am I?'")
    print()
    
    while True:
        userInput = input("User > ")

        if userInput == "exit":
            break

        history.add_user_message(userInput)

        result = await azure_chat_completion.get_chat_message_content(
            chat_history=history,
            settings=execution_settings,
            kernel=kernel,
        )

        print("Assistant > " + str(result))
        print()

        history.add_message(result)

# Run the chat
await main()